# work

This site was built with the following [configuration](#configuration), and may have generated some [logs](#logs).

In [ ]:
import json
import subprocess
from pathlib import Path

import importnb
import jinja2
from IPython.display import HTML, display
from nbconvert.filters import ansi2html, markdown2html

with importnb.Notebook():
    from jupyak.tasks import load_tasks
    from jupyak.tasks._yak import Repo, Yak

In [ ]:
HTML(
    """
<style>
.jpyk-log { width: 100% !important; }
.jpyk-log td { text-align: left !important; }
.jpyk-log th { text-align: left !important; }
.jpyk-log .elapsed { text-align: right !important; width: 5em;}
.jpyk-log .result { width: 5em;}
.jpyk-log th.name { width: 15em;}
details pre {padding: 1em !important;}
</style>
"""
)

In [ ]:
load_tasks()
yak = Yak(conf_dir=str(Path.cwd().parent))

In [ ]:
def ref_link(url: str, repo: Repo, attrs="", loop=None):
    ref = last = url.split("#")[0].split("?")[0].split("/")[-1]
    if "/pull/" in url:
        last = f"#{last}"
        ref = f"pr-{ref}"
    try:
        commit, log = (
            subprocess.check_output(
                ["git", "log", "-1", "--pretty=%H: %B", ref],
                cwd=str(repo.work_path),
                encoding="utf-8",
            )
            .splitlines()[0]
            .split(":", 1)
        )
    except Exception as err:
        print(err)
        log = "..."
        commit = ""
    cols = f"""
        <td {attrs}>
            <a href="{url}">{last}</a>
        </td>
        <td {attrs}>
            <a href="{repo.github.url}/commit/{commit}"><code>{commit[:7]}</code></a>
        </td>
        <td {attrs}>
        {log}
        </td>
    """

    if loop and loop.length > 1:
        if not loop.index0:
            cols += "</tr>"
        else:
            cols = "<tr>" + cols
    return cols

In [ ]:
YAK_TMPL = jinja2.Template(
    """
<h3>{{ yak.pr.title }}</h3>

{{ md2h(yak.pr.description) }}

<h4>repos</h4>

<table class="jpyk-log">
    <thead>
        <tr>
            <th rowspan="2">repo</th>
            <th colspan="3">baseline</th>
            <th colspan="3">merged with...</th>
        </tr>
        <tr>
            {% for i in range(2) %}
            <th>branch/ref</th>
            <th>commit</th>
            <th>log</th>
            {% endfor %}
        </tr>
    </thead>
    <tbody>
        {% for name, r in yak.repos.items() -%}{%- set gh = r.github -%}
        <tr>
            {% set attrs = "" %}
            {% if (gh.merge_with | length) > 1 %}
                {% set attrs = 'rowspan="' ~ (gh.merge_with | length) ~ '"' %}
            {% endif %}
            <th {{ attrs }}>
                <a href="{{r.url}}">{{r.name}}</a>
            </th>
            {{ ref(gh.baseline, r, attrs=attrs) -}}
            {% for m in gh.merge_with %}
                {{ ref(m, r, loop=loop) }}</li>
            {%- endfor -%}
        </tr>
        {% endfor %}
    </tbody>
</table>
"""
)

## configuration

In [ ]:
HTML(YAK_TMPL.render(yak=yak, ref=ref_link, md2h=markdown2html))

## logs

In [ ]:
LOGS = sorted(
    (Path.cwd() / "../work/build/logs/").resolve().glob("*.json"),
    key=lambda x: x.stat().st_mtime,
)

In [ ]:
TMPL = jinja2.Template(
    """
<h2>{{ log.stem }}</h2>
<table class="jpyk-log">
    <thead>
        <tr>
            {% for h in headers %}<th class="{{h}}">{{ h }}</th>{% endfor %}
            {% for f in fields %}<td class="{{f}}">{{ f }}</td>{% endfor %}
            {% for d in details %}<td class="{{d}}">{{ d }}</td>{% endfor %}
        </tr>
    </thead>
    <tbody>
        {% for t in tasks %}
        <tr>
            {% for h in headers %}
            <th >
                {% if h == "name" %}
                {{ t[h].replace("shave:", "") }}
                {% else %}
                {{ t[h] }}
                {% endif %}
            </th>
            {% endfor %}
            {% for f in fields %}<td class="{{ f }}">
                {% if f == "elapsed" and t[f] %}
                    {{ (t[f]) | round(1) }} s
                {% else %}
                    {{ t[f] or "" }}
                {% endif %}
            </td>
            {% endfor %}
            {% for d in details %}<td width="100%">
                {% if t[d] %}
                <details class="jp-RenderedText">
                    <summary>{{ t[d] | count }} lines</summary>
                    <pre>{{ ansi2html("".join(t[d])) }}</pre>
                </details>
                {% endif %}
            </td>{% endfor %}
        </tr>
        {% endfor %}
    </tbody>
</table>
"""
)

In [ ]:
HEADERS = ["result", "name"]
FIELDS = ["elapsed"]
DETAILS = ["out", "err"]
CONTEXT = dict(headers=HEADERS, fields=FIELDS, details=DETAILS, ansi2html=ansi2html)

In [ ]:
for log in LOGS:
    try:
        tasks = [
            t
            for t in json.loads(log.read_text(encoding="utf-8"))["tasks"]
            if t["elapsed"]
        ]
    except:
        continue
    display(HTML(TMPL.render(log=log, tasks=tasks, **CONTEXT)))